### Log Ratio Method for Identifying Tree Loss

#### Import Packages

In [1]:
# keep logging quiet
import logging

logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [2]:
# import packages
import ipywidgets
import ipyleaflet
import numpy as np
import matplotlib.pyplot as plt
import descarteslabs.workflows as wf
from utils import FullArray, DrawControl


In [9]:

# Before Composite
before = (
    wf.ImageCollection.from_id(
        "sentinel-1:GRD", start_datetime="2016-01-01", end_datetime="2016-12-31"
    )
    .pick_bands("vh")
    .median(axis="images")
)
before.visualize("Sentinel-1 2016 composite", colormap="Greens", scales=[[0, 0.09]])

# After Composite
after = (
    wf.ImageCollection.from_id(
        "sentinel-1:GRD", start_datetime="2019-01-01", end_datetime="2019-12-31"
    )
    .pick_bands("vh")
    .median(axis="images")
)
after.visualize("Sentinel-1 2019 composite", colormap="Greens", scales=[[0, 0.09]])

# Take the log of the ratio of both composites (equivalent to difference of logs)
log_ratio = wf.log10(before / after)

# Define a threshold for the log ratio
threshold = 0.01

# Threshold the log ratio layer
deforestation = log_ratio > threshold

#### Morpholigical Filtering

In [10]:
# Define simple functions for erosion and dilation


def erode_op(map_layer, iters, kernel):
    map_layer = ~map_layer
    for i in range(iters):
        map_layer = wf.conv2d(map_layer, kernel) > 0
    map_layer = ~map_layer
    return map_layer


def dilate_op(map_layer, iters, kernel):
    for i in range(iters):
        map_layer = map_layer * 1.0
        map_layer = wf.conv2d(map_layer, kernel) > 0
    return map_layer


# Define a kernel and perform one erosion followed by two dilations
kernel = wf.Kernel(dims=(3, 3), data=[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0])

eroded = erode_op(deforestation, iters=1, kernel=kernel)
dilated = dilate_op(eroded, iters=2, kernel=kernel)

# Visualize the resulting deforestation mask
lyr = dilated.mask(dilated == 0).visualize(
    "Deforestation", checkerboard=False, colormap="bwr"
)
lyr.opacity = 0.7

#### Visualise Imagery

In [12]:
# Location
location = 'CongoKazumba' # Name for saved .pngs
lat, lon = -6.42, 22.3983 # Center of Area of Interest 
zoom = 15 # Map tile zoom, 16 default

#initialise interactive map
m = wf.map.map
m1 = wf.interactive.MapApp()
m1.center, m1.zoom = (lat, lon), zoom
m.layout.width = "1200px"
m.layout.height = "900"

#define marker for deforestation identification
dc = DrawControl(circlemarker={}, rectangle={}, polyline={})
wf.map.add_control(dc)

# Interactively compute deforested acreage
pixel_sum = FullArray(wf.map, variable=dilated, draw_control=dc)

output = ipywidgets.Output()


@output.capture()
def print_acreage(*args, **kwargs):
    # Estimate the surface area of one pixel in hectares
    pix_size = (156543.00 / 2 ** (max(m.zoom, 0))) ** 2 / 10000
    # Get the array equivalent to the polygon drawn by the user
    d = pixel_sum.data
    # Compute deforested and total acreage
    deforested = round((d == 1).sum() * pix_size, 2)
    total = round(((d == 1) | (d == 0)).sum() * pix_size, 2)
    print(
        "Deforested area within AOI : \n\n{} ha (of {} ha in total)".format(
            deforested, total
        )
    )


# Perform the acreage computation
dc.on_draw(print_acreage)

# Print acreage in a clear button on the map in the top right corner
output_clear_button = ipywidgets.Button(
    description="Clear Output Window", layout=ipywidgets.Layout(width="auto")
)
acreage_button = ipywidgets.Button(
    dc.on_draw, description="Acreage", layout=ipywidgets.Layout(width="auto")
)
output_clear_button.on_click(lambda b: output.clear_output())
output_vbox = ipywidgets.VBox([output, output_clear_button])
m.add_control(ipyleaflet.WidgetControl(widget=output_vbox, position="bottomleft"))

ipywidgets.HBox([wf.map])

TypeError: __init__() takes 1 positional argument but 2 were given